In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import random
import igraph as ig
import os



def read_dimacs_graph(filename):
    edges = []
    weights = []
    max_node = 0
    
    with open(filename, "r") as f:
        for line in f:
            if line.startswith("a"):
                _, u, v, w = line.strip().split()
                u, v, w = int(u), int(v), int(w)
                edges.append((u - 1, v - 1))
                weights.append(w)
                max_node = max(max_node, u, v)


    G = ig.Graph()
    G.add_vertices(max_node)
    G.add_edges(edges)
    G.es["weight"] = weights
    return G


def save_dimacs_graph(G: ig.Graph, filename, comment: str = ""):
    with open(filename, "w") as f:
        if not comment.isspace():
            f.write(f"c {comment}\n")
        f.write(f"p sp {G.vcount()} {G.ecount()}\n")
        for e in G.es:
            u, v = G.es[e.index].tuple
            w = e["weight"] if "weight" in e.attributes() else 1
            f.write(f"a {u + 1} {v + 1} {w}\n")

In [4]:
def remove_edges(G: ig.Graph, remove_percent=0.1):
    assert 0 < remove_percent < 1

    remove_edges_count = int(G.ecount() * remove_percent)
    # print(f"Will be removed: {remove_edges_count} edges.")

    h_edges = list(G.es)

    bridge_ids = set(G.bridges())
    sorted_edges = [
    (e.source, e.target, e["weight"] if "weight" in e.attributes() else 1)
    for e in h_edges
    if e.index not in bridge_ids
    ]
    sorted_edges = sorted(sorted_edges, key=lambda x: x[2])

    result_graph: ig.Graph = G.copy()

    degrees = {v.index: deg for v, deg in zip(result_graph.vs, result_graph.degree())}
    edges_to_remove = set()

    while len(edges_to_remove) < remove_edges_count and sorted_edges:
        u, v, _ = sorted_edges.pop()

        if degrees[u] <= 1 or degrees[v] <= 1:
            continue

        degrees[u] -= 1
        degrees[v] -= 1

        edges_to_remove.add((u, v))

    result_graph.delete_edges(edges_to_remove)

    # print(f"Was: {G.ecount()}, Now: {result_graph.ecount()} (edges). It is {(G.ecount() - result_graph.ecount()) / G.ecount() * 100:.2f}% of the original number.")
    return result_graph

# Experiment 1 (Density)

In [12]:
def calc_density(G: ig.Graph):
    return (2 * G.ecount()) / (G.vcount() * (G.vcount() - 1))


# Experiment 1
exp_dataset = {
    "San Francisco Bay Area": "USA-road-d.BAY.gr",
    "California and Nevada": "USA-road-d.CAL.gr",
    "Colorado": "USA-road-d.COL.gr",
    "Florida": "USA-road-d.FLA.gr",
    "Great Lakes": "USA-road-d.LKS.gr",
    "Northeast USA": "USA-road-d.NE.gr",
    "Northwest USA": "USA-road-d.NW.gr",
    "New York City": "USA-road-d.NY.gr",
}

PERCENTS = [0.01, 0.03, 0.05, 0.07, 0.1, 0.13, 0.15]

for name, filename in tqdm.tqdm(exp_dataset.items()):
    G = read_dimacs_graph(filename)

    for percent in PERCENTS:
        G2 = remove_edges(G, percent)

        assert len(list(G2.connected_components(mode="strong"))) == 1

        new_name = (
            os.path.splitext(os.path.basename(filename))[0]
            + f".perc_{int(percent * 100)}.gr"
        )

        save_dimacs_graph(
            G2,
            f"dataset/{new_name}",
            comment=f"Name: {name}, Filename: {filename} Density: {calc_density(G2)}, Start density: {calc_density(G)}",
        )


100%|██████████| 8/8 [10:09<00:00, 76.14s/it] 


# Experiment 2 (Weights)

In [9]:
# Randomize weights
for name, filename in tqdm.tqdm(exp_dataset.items()):
    G = read_dimacs_graph(filename)
    G.es['weight'] = [random.randint(1, 100) for _ in G.es]

    new_name = (
        os.path.splitext(os.path.basename(filename))[0]
        + f".random.gr"
    )

    save_dimacs_graph(
        G,
        f"dataset/{new_name}"
    )

100%|██████████| 8/8 [01:21<00:00, 10.18s/it]


In [10]:
# Shuffle weights
for name, filename in tqdm.tqdm(exp_dataset.items()):
    G = read_dimacs_graph(filename)
    weights = G.es['weight'][:]
    random.shuffle(weights)

    G.es['weight'] = weights

    new_name = (
        os.path.splitext(os.path.basename(filename))[0]
        + f".shuffle.gr"
    )

    save_dimacs_graph(
        G,
        f"dataset/{new_name}"
    )

100%|██████████| 8/8 [01:26<00:00, 10.81s/it]
